In [ ]:
import numpy as np
import OpenEXR as exr
import Imath

def readEXR(filename, convert_to_sRGB = False):
    """Read color + depth data from EXR image file.

    Parameters
    ----------
    filename : str
        File path.

    Returns
    -------
    img : RGB or RGBA image in float32 format. Each color channel
          lies within the interval [0, 1].
          Color conversion from linear RGB to standard RGB is performed
          internally. See https://en.wikipedia.org/wiki/SRGB#The_forward_transformation_(CIE_XYZ_to_sRGB)
          for more information.

    Z : Depth buffer in float32 format or None if the EXR file has no Z channel.
    """

    exrfile = exr.InputFile(filename)
    header = exrfile.header()

    dw = header['dataWindow']
    isize = (dw.max.y - dw.min.y + 1, dw.max.x - dw.min.x + 1)

    channelData = dict()

    # convert all channels in the image to numpy arrays
    for c in header['channels']:
        C = exrfile.channel(c, Imath.PixelType(Imath.PixelType.FLOAT))
        C = np.fromstring(C, dtype=np.float32)
        C = np.reshape(C, isize)

        channelData[c] = C

    colorChannels = ['R', 'G', 'B', 'A'] if 'A' in header['channels'] else ['R', 'G', 'B']
    img = np.concatenate([channelData[c][...,np.newaxis] for c in colorChannels], axis=2)

    # linear to standard RGB
    if convert_to_sRGB:
      img[..., :3] = np.where(img[..., :3] <= 0.0031308,
                            12.92 * img[..., :3],
                            1.055 * np.power(img[..., :3], 1 / 2.4) - 0.055)

    # sanitize image to be in range [0, 1]
    img = np.where(img < 0.0, 0.0, np.where(img > 1.0, 1, img))

    #I dont need Z, discarded
    #Z = None if 'Z' not in header['channels'] else channelData['Z']

    return img

Okay, Ima try this again. I will Recalculate the gram matrices, this time useing torch instead of tf
I believe I made some crucial mistakes last time

Todo:


*   Adapt code, its for style transfer now, dont need a lot of whats going on
**  Remove content layer stuff
**  check if module/layer variables can be read after something got pushed through the network
*   After gram generation, check value space, if nescessary normalize (I think it might not be nescessary this time)
*



In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models
import os
import copy

from tqdm import tqdm  # For nice progress bar!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# desired size of the output image
imsize = 224

loader = transforms.Compose([
    #transforms.Resize([imsize, imsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor

def image_loader(image_name):
    if (image_name.endswith(".exr")):
      image = readEXR(image_name)
      #image = Image.fromarray(np.uint8(image)).convert('RGB')
    else:
      image = Image.open(image_name).convert('RGB')

    #print(image)
    #image.convert('RGB')
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)[:,:3,16:240,16:240]

    #print("shape" + str(image.shape))
    return image.to(device, torch.float)

In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.figure()
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated

#plt.figure()
#imshow(style_img, title='Style Image')

#plt.figure()
#imshow(content_img, title='Content Image')

In [ ]:
def gram_matrix(input):

    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    return G.div(c * d) # I still dont quite understand why this normalization is done. but it does help to get the values somewhat closer together.

In [ ]:
# class StyleLoss(nn.Module):

#     def __init__(self, target_feature):
#         super(StyleLoss, self).__init__()
#         self.target = gram_matrix(target_feature).detach()

#     def forward(self, input):
#         self.gram = gram_matrix(input)
#         self.loss = F.mse_loss(G, self.target)
#         return input

In [ ]:
class StyleExtract(nn.Module):

    def __init__(self):
        super(StyleExtract, self).__init__()

    def forward(self, input):
        self.gram = gram_matrix(input)
        return input

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
#@title Building the model that with layers that produce gram matrices.
#@markdown Iterates over the prebuilt vgg network, makes changes if nescessary (change "layer" to whatever we want to insert in iteration (see Relu)) <br>(**TODO: change max pooling to avg pooling?**). <br>
#@markdown if the layer was one of the style layers we enter our custom style layer in between. that layer does not actually do anything to the data in the network, but gives us a way to extract the style matrix after the image ran though the network
#@markdown Returnes the model and an array to access style matrices.
# desired depth layers to compute style/content losses :


#content_layers_default = ['conv_4']
style_layers_default = ['conv1_2', 'conv2_2', 'conv3_4', 'conv4_4', 'conv5_4']

def get_style_model_and_style_extract_layers(cnn, normalization_mean, normalization_std,
                               #style_img, content_img,
                               #content_layers=content_layers_default,
                               style_layers=style_layers_default):
    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    #content_losses = []
    style_grams = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 1  # increment every time we see a conv resets when pooling
    j = 1  # increase every time we see a pooling
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            name = 'conv{}_{}'.format(j,i)
            i += 1
        elif isinstance(layer, nn.ReLU):
            name = 'relu{}_{}'.format(j,i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool{}_{}'.format(j,i)
            layer = nn.AvgPool2d(3) #@markdown changed to avg pool here like Match did in their paper
            j += 1
            i = 1
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn{}_{}'.format(j,i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in style_layers:
            style_extract = StyleExtract()
            model.add_module("-----------------style_extract_{}".format(j), style_extract)
            style_grams.append(style_extract)

            if name == style_layers[-1]:
              break

    # now we trim off the layers after the last content and style losses
    # for i in range(len(model) - 1, -1, -1):
    #     if isinstance(model[i], StyleExtract):
    #         break

    #model = model[:(i + 1)]

    return model, style_grams #, content_losses

In [ ]:
model, styles = get_style_model_and_style_extract_layers(cnn, cnn_normalization_mean, cnn_normalization_std)

C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/1744401072.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean).view(-1, 1, 1)
C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/1744401072.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std).view(-1, 1, 1)


In [ ]:
model

Sequential(
  (0): Normalization()
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU()
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (-----------------style_extract_1): StyleExtract()
  (relu1_3): ReLU()
  (pool1_3): AvgPool2d(kernel_size=3, stride=3, padding=0)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU()
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (-----------------style_extract_2): StyleExtract()
  (relu2_3): ReLU()
  (pool2_3): AvgPool2d(kernel_size=3, stride=3, padding=0)
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU()
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU()
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_4): ReLU()
  (conv3_4): Conv2d(256, 256, kernel_siz

In [ ]:
loadDirectory = r'C:\Users\mwinkelmueller\Dropbox\Masl Stuff\BachelorArbeit\Datasets\BiggerShaderDataset\Renders'
saveDirectory = r'C:\Users\mwinkelmueller\Dropbox\Masl Stuff\BachelorArbeit\Datasets\BiggerShaderDataset\Grams'




grams = {}

mins = [float('inf')] * 5
maxs = [float('-inf')] * 5
hist = []

model, styles = get_style_model_and_style_extract_layers(cnn, cnn_normalization_mean, cnn_normalization_std)

count = 0

print("Generating Grams")
for filename in tqdm(os.listdir(loadDirectory)):

  if not filename.endswith(".exr"):
    continue

  loadPath = os.path.join(loadDirectory,filename)

  im = image_loader(loadPath)

  with torch.no_grad():
    model(im)

  for i, style in enumerate(styles):    #why the fuck did i do it this ugly way. why. i need a break
    gram = style.gram.cpu()
    #grams[filename + str(i)] = gram
    maxs[i] = max(maxs[i], torch.max(gram))
    mins[i] = min(mins[i], torch.min(gram))
    hist[i] += np.histogram(gram,bins=10)


  torch.cuda.empty_cache()


print(maxs)
print(mins)




C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/1744401072.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean).view(-1, 1, 1)
C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/1744401072.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std).view(-1, 1, 1)


Generating Grams


  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/2045026391.py:35: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  C = np.fromstring(C, dtype=np.float32)
100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [1:46:15<00:00, 15.68it/s]

[tensor(637.8943), tensor(554.4865), tensor(405.1444), tensor(45.7615), tensor(4.8098)]
[tensor(-617.9616), tensor(-253.7502), tensor(-148.7658), tensor(-23.1019), tensor(-2.9467)]


[tensor(637.8943), tensor(554.4865), tensor(405.1444), tensor(45.7615), tensor(4.8098)]

[tensor(-617.9616), tensor(-253.7502), tensor(-148.7658), tensor(-23.1019), tensor(-2.9467)]

In [ ]:
print("normalising and saving Grams")

#maxs = torch.tensor([637.8943, 554.4865, 405.1444, 45.7615, 4.8098])
#mins = torch.tensor([-617.9616, -253.7502, -148.7658, -23.1019, -2.9467])


#for filename in tqdm(os.listdir(loadDirectory)):
for filename in tqdm(os.listdir(loadDirectory)):

  concatMat = torch.empty(0)

  if not filename.endswith(".exr"):
    continue

  loadPath = os.path.join(loadDirectory,filename)
  savePath = os.path.join(saveDirectory,filename + "_ConcatGram")

  im = image_loader(loadPath)

  with torch.no_grad():
    model(im)


  for i, style in enumerate(styles):
    matrix = style.gram.cpu()
    matrix = matrix.subtract(mins[i])
    matrix = matrix.div(maxs[i] - mins[i])

    if torch.max(matrix) > 1 or torch.min(matrix) < 0:
      print("ALAAARRMM")

    concatMat = torch.cat([concatMat, torch.flatten(matrix)], dim=0)




  torch.save(concatMat, savePath)

  torch.cuda.empty_cache()

print(concatMat.shape)

normalising and saving Grams


  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]C:\Users\MWINKE~1\AppData\Local\Temp/ipykernel_14824/2045026391.py:35: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  C = np.fromstring(C, dtype=np.float32)
100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [1:35:34<00:00, 17.44it/s]

torch.Size([610304])


# Testing shenanigans

In [ ]:
from operator import itemgetter
maxItem = [0] * 5
minMaxItem = [999999999] * 5
minItem = [10000000000] * 5
maxMinItem = [-999999] * 5
#allGrams[3]["gram1"]

for grams in tqdm(allGrams):

  for i in range(5):
    maxItem[i] =    max(maxItem[i], grams['gram'+str(i)+ " max"])
    minMaxItem[i] = min(minMaxItem[i], grams['gram'+str(i)+ " max"])
    minItem[i] =    min(minItem[i], grams['gram'+str(i)+ " min"])
    maxMinItem[i] = max(maxMinItem[i], grams['gram'+str(i)+ " min"])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14855.71it/s]


In [ ]:
print(maxItem  )
print(minMaxItem)
print(minItem  )
print(maxMinItem)

[tensor(33.1186), tensor(28.9133), tensor(138.9145), tensor(25.5923), tensor(3.6506)]
[tensor(5.2760), tensor(8.1205), tensor(33.2691), tensor(4.3334), tensor(2.6855)]
[tensor(-28.2296), tensor(-24.2494), tensor(-62.1443), tensor(-10.1801), tensor(-2.6729)]
[tensor(-4.0909), tensor(-4.1236), tensor(-8.7225), tensor(-1.3851), tensor(-1.1448)]


In [ ]:
test = []

test.append(4)
print(test)

In [ ]:
im = image_loader('/content/image.exr')
model(im)

In [ ]:
for style in styles:

  print("min = {}, max = {}, diff= {}".format(torch.min(style.gram), torch.max(style.gram), torch.min(style.gram) - torch.max(style.gram)))
  imshow(style.gram)


In [ ]:
for style in styles:


  show = style.gram.add(-torch.min(style.gram))
  show = show.div(torch.max(show))
  print("min = {}, max = {}, diff= {}".format(torch.min(show), torch.max(show), torch.min(show) - torch.max(show)))
  imshow(show)
  plt.figure()
  hist = [0] * 100
  thisHist = np.histogram(show.detach(),bins=10,range=[0,1])
  print(thisHist[0])
  plt.bar(thisHist[1][:10],thisHist[0][:10])

In [ ]:
hist = [0] * 100
thisHist = np.histogram(data,bins=100,range=[-10,20])
plt.bar(thisHist[1][:100],hist[:100])


In [ ]:
show.save()

In [ ]:


  # #path = os.path.join(directory,filename)
path = '/content/image.exr'

image = image_loader(path)

imshow(image)


In [ ]:
img.shape